# Jesse's build problem

2015-08-10 Adam Lyon

Jesse is having a build problem where he's getting a library not found error when he builds gm2geom, puts it in the  local products area, and then tries to build gm2ringsim. Trying myself.

In [9]:
%%file iterm-jesseBuildProblem.scr
tell application "iTerm"
    activate
    set myterm to (make new terminal)
    tell myterm
        launch session "Default Session"
        tell the last session   
            set name to "temp-JesseBuildError-prof"
            write text "source /cvmfs/oasis.opensciencegrid.org/gm2/prod/g-2/setup"
            write text "cd ~/Development/g-2/temp/jesseBuildError/"
            write text "source localProducts_gm2_v6_01_00_prof/setup"
            write text "setup ninja v1_5_3a"
            write text ". mrb s"
        end tell
    end tell
end tell

Overwriting iterm-jesseBuildProblem.scr


In [10]:
!osascript iterm-jesseBuildProblem.scr

I've checked out `gm2geom`, built and installed it. 

Building `gm2ringsim` against the local product `gm2geom` 

I see the same problem as Jesse!!

```
[23/203] Linking CXX shared library gm2ringsim/lib/libgm2ringsim_fields_StorageFieldController_service.dylib
FAILED:
   ld: library not found for -lgm2geom_common_Gm2Constants_service
```

Why is there a `-l` in front? All of the other libraries are explicitly listed in the `ld` call.

Looking at `CmakeCache.txt`, there's no mention of this library while other libraries are mentioned.

Looks like a `cmake` problem. May have to get my own cetbuildtools. Ugh

The Gm2Constants library is requested by `gm2ringsim_fields_StorageFieldController_service.dir/StorageFieldController_service.cc`

See `srcs/gm2ringsim/fields`. The `CMakeLists.txt` says,
```
#art_make( LIB_LIBRARIES gm2ringsim_fields)
art_make(
          SERVICE_LIBRARIES
          gm2geom_common_Gm2Constants_service
          gm2geom_inflector_InflectorGeometry_service
          ${G4_LIB_LIST}
          artg4_util
)
install_headers()
<more>
```

Here's the `cmake` command that is actually run (from `mrb b -C -v --generator ninja`)
```
eval CC=gcc CXX=g++ FC=gfortran cmake -DCMAKE_INSTALL_PREFIX=/Users/lyon/Development/g-2/temp/jesseBuildError/localProducts_gm2_v6_01_00_prof -DCMAKE_BUILD_TYPE=Prof -DCET_TEST_GROUPS= -GNinja /Users/lyon/Development/g-2/temp/jesseBuildError/srcs
```

I think the actual problem is in the `find_ups_product(gm2geom v3_01_00)`, because that's what sets the library list.

We're using cetbuildtools v4_08_01 - copied to localproducts

I turned on some debug in `find_ups_products` and see,
```
find_ups_product: found a cmake configure file in /Users/lyon/Development/g-2/temp/jesseBuildError/localProducts_gm2_v6_01_00_prof/gm2geom/v3_01_00/d14.x86_64.e7.prof/lib/gm2geom/cmake/gm2geomConfig.cmake
```

That configure file is missing a `cet_find_library` line for GM2GEOM_COMMON_GM2CONSTANTS_SERVICE. Why?

So the real problem is in building `gm2geom` !!

Looking at `cetbuildtools/.../Modules/CetMakeConfig.cmake` . The problem is that `$CONFIG_LIBRARY_LIST` is missing our library.

`CONFIG_LIBRARY_LIST` gets constructed by calls to `cet_add_to_library_list` in `CetCMakeUtils.cmake`. This is called by (among others) `cet_make_library` in `CetMake.cmake`. Not sure who calls this. Will try debugging `cet_make_library`.

I don't see `cet_make_library` called for gm2geom_common_Gm2Constants_service. I don't see the other services here either. -- red herring

I instrumented `BasicPlugin.cmake` which also calls `cet_add_to_library_list`. I see the services in here! I see,

```
-- Calling cet_add_to_library_list for gm2geom_inflector_InflectorGeometry_service
-- Calling cet_add_to_library_list for gm2geom_calo_CaloGeometry_service
-- Calling cet_add_to_library_list for gm2geom_Core_Geometry_service
-- Calling cet_add_to_library_list for gm2geom_common_Gm2Constants_service
```

Hey - I see Gm2Constants service! 

So I see it being added! Something then must be removing it.

Hmmmm! I see `gm2geom_common_Gm2Constants_service` added to `CONFIG_LIBRARY_LIST` **after** `cet_make_config` is called, so it's too late.

```
-- Calling cet_add_to_library_list for gm2geom_Core_Geometry_service
Adding gm2geom_Core_Geometry_service to CONFIG_LIBRARY_LIST
CONFIG_LIBRARY_LIST is gm2geom_strawtracker;gm2geom_world;gm2geom_vac;gm2geom_station;gm2geom_quad;gm2geom_kicker;gm2geom_inflector;gm2geom_inflector_InflectorGeometry_service;gm2geom_fiberHarp;gm2geom_collimator;gm2geom_calo;gm2geom_calo_CaloGeometry_service;gm2geom_arc;gm2geom_Core_Geometry_service
-- cet_make_config: CONFIG_LIBRARY_LIST is  <--------*** HERE 
gm2geom_strawtracker;gm2geom_world;gm2geom_vac;gm2geom_station;gm2geom_quad;gm2geom_kicker;gm2geom_inflector;gm2geom_inflector_InflectorGeometry_service;gm2geom_fiberHarp;gm2geom_collimator;gm2geom_calo;gm2geom_calo_CaloGeometry_service;gm2geom_arc;gm2geom_Core_Geometry_service
-- Calling cet_add_to_library_list for gm2geom_common_Gm2Constants_service
Adding gm2geom_common_Gm2Constants_service to CONFIG_LIBRARY_LIST
CONFIG_LIBRARY_LIST is gm2geom_strawtracker;gm2geom_world;gm2geom_vac;gm2geom_station;gm2geom_quad;gm2geom_kicker;gm2geom_inflector;gm2geom_inflector_InflectorGeometry_service;gm2geom_fiberHarp;gm2geom_collimator;gm2geom_calo;gm2geom_calo_CaloGeometry_service;gm2geom_arc;gm2geom_Core_Geometry_service;gm2geom_common_Gm2Constants_service
```

I see the problem!!!

The problem is that the main `CMakeLists.txt` file has:

```
# Build these directories
add_subdirectory( strawtracker )
add_subdirectory( world )
add_subdirectory( vac )
add_subdirectory( station )
add_subdirectory( quad )
add_subdirectory( kicker )
add_subdirectory( inflector )
add_subdirectory( fiberHarp )
add_subdirectory( collimator )
add_subdirectory( calo )
add_subdirectory( arc )
add_subdirectory( Core )
add_subdirectory( ups )
add_subdirectory( test )
add_subdirectory( common )
```

The `ups` line needs to be last!!  

Fixing that makes things work -- now `GM2GEOM_COMMON_GM2CONSTANTS_SERVICE` is in the `gm2geomConfig.cmake` file